In [1]:
import librosa as librosa
import numpy as np
from scipy import signal
import random
import time
import torch.nn as nn
from numba import njit, cuda, set_num_threads

from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
import sounddevice as sd
import torch.optim as optim

import plotly.express as px

from torch.utils.data import DataLoader
import torch.utils.data as data
from torchvision import transforms, datasets
import torch

c:\Users\p3tro\anaconda3\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Users\p3tro\anaconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
def get_noisy_sinusoid(freq, amplitude, noise_amplitude, length):
    t = np.linspace(0, length, length * 10 ** 3, endpoint=False)
    x = amplitude * np.sin(2 * np.pi * freq * t)
    noise = np.random.normal(0, noise_amplitude, len(x))
    for i in range(len(x)):
        noise[i] += (random.randint(1, 200) // 200) * (-1) ** (random.randint(1,2))
    return t, x, x + noise

In [3]:
t, clear_signal, noisy_signal = get_noisy_sinusoid(10, 1, 0.1, 100)

In [32]:
def get_noisy_sinusoid_array(freq, amplitude, noise_amplitude, length, amount=10):
    clear_signals = []
    noisy_signals = []
    
    for i in tqdm(range(amount)):
        _, x, y = get_noisy_sinusoid(freq, amplitude, noise_amplitude, length)
        
        clear_signals.append(x)
        noisy_signals.append(y)

    return clear_signals, noisy_signals

class SinusoidDataset(data.Dataset):
    def __init__(self, freq, amplitude, noise_amplitude, length, amount):
        self.input, self.output = get_noisy_sinusoid_array(freq, amplitude, noise_amplitude, length, amount)

    def __getitem__(self, index):
        input = self.input[index]
        output = self.output[index]

        return input, output

    def __len__(self):
        return len(self.input)

In [34]:
dataset = SinusoidDataset(10, 1, 0.1, 100, 1)

100%|██████████| 1/1 [00:00<00:00,  4.50it/s]


In [38]:
class Autoencoder(nn.Module):
    def __init__(self, input_size):
        super(Autoencoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv1d(16, 8, kernel_size=3, stride=2, padding=1),
            nn.ReLU()
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(8, 16, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose1d(16, 1, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

input_size = 100
hidden_size = 128
learning_rate = 0.01
batch_size = 32
num_epochs = 100

model = Autoencoder(input_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [44]:
dataset = SinusoidDataset(10, 1, 0.1, input_size, amount=100)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

100%|██████████| 100/100 [00:21<00:00,  4.64it/s]


In [45]:
def train_autoencoder(model, dataloader, criterion, optimizer, num_epochs):
    model.train()
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        for signals in dataloader:
            optimizer.zero_grad()
            
            # Розпакування звукових сигналів
            clean_signals, noisy_signals = signals
            
            # Перетворення на тензори PyTorch
            clean_signals = clean_signals.unsqueeze(1).float()
            noisy_signals = noisy_signals.unsqueeze(1).float()
            
            # Пропуск зашумлених сигналів через автоенкодер
            outputs = model(noisy_signals)
            
            # Обчислення функції втрати
            loss = criterion(outputs, clean_signals)
            
            # Зворотнє поширення помилки та оптимізація
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        # Виведення середньої втрати на епоху
        epoch_loss = running_loss / len(dataloader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

In [46]:
train_autoencoder(model, dataloader, criterion, optimizer, num_epochs)

Epoch [1/100], Loss: 0.0026
Epoch [2/100], Loss: 0.0025
Epoch [3/100], Loss: 0.0025
Epoch [4/100], Loss: 0.0025
Epoch [5/100], Loss: 0.0024
Epoch [6/100], Loss: 0.0024
Epoch [7/100], Loss: 0.0024
Epoch [8/100], Loss: 0.0023
Epoch [9/100], Loss: 0.0023
Epoch [10/100], Loss: 0.0023
Epoch [11/100], Loss: 0.0023
Epoch [12/100], Loss: 0.0022
Epoch [13/100], Loss: 0.0022
Epoch [14/100], Loss: 0.0022
Epoch [15/100], Loss: 0.0022
Epoch [16/100], Loss: 0.0022
Epoch [17/100], Loss: 0.0022
Epoch [18/100], Loss: 0.0021
Epoch [19/100], Loss: 0.0021
Epoch [20/100], Loss: 0.0021
Epoch [21/100], Loss: 0.0021
Epoch [22/100], Loss: 0.0021
Epoch [23/100], Loss: 0.0021
Epoch [24/100], Loss: 0.0021
Epoch [25/100], Loss: 0.0020
Epoch [26/100], Loss: 0.0020
Epoch [27/100], Loss: 0.0020
Epoch [28/100], Loss: 0.0020
Epoch [29/100], Loss: 0.0020
Epoch [30/100], Loss: 0.0020
Epoch [31/100], Loss: 0.0020
Epoch [32/100], Loss: 0.0020
Epoch [33/100], Loss: 0.0020
Epoch [34/100], Loss: 0.0020
Epoch [35/100], Loss: 0

In [50]:
clear_signal, noisy_signal = dataset[0]
noisy_signal_tensor = torch.from_numpy(noisy_signal).unsqueeze(0).unsqueeze(1).float()
filtered_signal = model(noisy_signal_tensor).squeeze().detach().numpy()

In [52]:
fig = px.line(x=t[:1000], y=noisy_signal[:1000], title='Noisy signal')
fig.add_scatter(x=t[:1000], y=filtered_signal[:1000], mode='lines', name='filtered signal')
fig.add_scatter(x=t[:1000], y=clear_signal[:1000], mode='lines', name='clear signal')
fig.show()